In [0]:
# Required each time the cluster is restarted which should be only on the first notebook as they run in order
tiers = ["bronze","silver","gold"]
adls_paths = {tier: f"abfss://{tier}@kbbsdedbeefefeb.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[FileInfo(path='abfss://gold@kbbsdedbeefefeb.dfs.core.windows.net/earthquake_events_gold/', name='earthquake_events_gold/', size=0, modificationTime=1744633781000)]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try: 
    #Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features',[])

    if not data:
        print("No data returned for the specified data range")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 364880 bytes.
Data successfully saved to abfss://bronze@kbbsdedbeefefeb.dfs.core.windows.net//2025-04-13_earthquake.json


In [0]:
data[1]

{'type': 'Feature',
 'properties': {'mag': 0.76,
  'place': '6 km NNW of The Geysers, CA',
  'time': 1744587731630,
  'updated': 1744589841298,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75164886',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75164886&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 9,
  'net': 'nc',
  'code': '75164886',
  'ids': ',nc75164886,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,scitech-link,',
  'nst': 12,
  'dmin': 0.01428,
  'rms': 0.02,
  'gap': 61,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 0.8 - 6 km NNW of The Geysers, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-122.806663513184, 38.8191680908203, 3.24000000953674]},
 'id': 'nc75164886'}

In [0]:
# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Return the dictionary directly
dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)
